In [ ]:
!pip install Flask pyngrok line-bot-sdk requests openai

## 如果是在Google Colab上執行，請先執行以下程式碼

In [ ]:
from google.colab import userdata

ngrok_key = userdata.get('ngrok_key')
line_access_token = userdata.get('line_access_token')
line_secret = userdata.get('line_secret')
port = 5000

In [ ]:
import os
from pyngrok import ngrok

In [ ]:
ngrok.set_auth_token(ngrok_key)
tunnel = ngrok.connect(5000, name="linebot_tunnel")
print(tunnel.public_url)

In [ ]:
from flask import Flask, request, abort

from linebot.v3 import (
    WebhookHandler
)
from linebot.v3.exceptions import (
    InvalidSignatureError
)
from linebot.v3.messaging import (
    Configuration,
    ApiClient,
    MessagingApi,
    ReplyMessageRequest,
    TextMessage,    
)

from linebot.v3.webhooks import (
    MessageEvent,
    TextMessageContent,
)

app = Flask(__name__)

configuration = Configuration(access_token=line_access_token)
handler = WebhookHandler(line_secret)


@app.route("/", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    print("BODY: ", body)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        app.logger.info("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'


@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    print("EVENT: ", event)
    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)
        line_bot_api.reply_message_with_http_info(
            ReplyMessageRequest(
                reply_token=event.reply_token,
                messages=[TextMessage(text=event.message.text),
                            TextMessage(text=event.message.text)]
            )
        )

if __name__ == "__main__":
    app.run(port=port)

In [ ]:
ok.disconnect(tunnel.public_url)